In [1]:
import os
from gtts import gTTS
import openai
import requests
import json
from time import sleep
import vlc

In [2]:
openai.api_key = ''

In [3]:
questions = []

In [4]:
question_obj = {}
question_obj['q_txt'] = 'What are your strengths and weaknesses?'
question_obj['id'] = 'question_1'
question_obj['lang'] = 'en'
question_obj['location'] = ''
question_obj['q_audio_f_name'] = 'q_'+str(question_obj['id'])+'.mp4'
question_obj['ans_audio_f_name'] = 'a_'+str(question_obj['id'])+'.mp4'
question_obj['ans_txt']=''
questions.append(question_obj)
question_obj = {}
question_obj['q_txt'] = 'What is your name?'
question_obj['id'] = 'question_2'
question_obj['lang'] = 'en'
question_obj['location'] = ''
question_obj['q_audio_f_name'] = 'q_'+str(question_obj['id'])+'.mp4'
question_obj['ans_audio_f_name'] = 'a_question_1.mp4'
question_obj['ans_txt']=''
questions.append(question_obj)

In [5]:
def convert_text_to_audio(question):
    myobj = gTTS(text=question['q_txt'], lang=question['lang'], slow=False)
    # Saving the converted audio in a mp3 file named
    myobj.save(question['location'] + question['q_audio_f_name'])

In [6]:
def convert_audio_to_text(question):
    #Loading answer from file
    file = open(question['location'] + question['ans_audio_f_name'], "rb")
    transcription = openai.Audio.transcribe("whisper-1", file)
    file.close()
    return transcription['text']

In [7]:
prefix = ' extract context and evaluate and rate 0 to 10 the answer '
interfix = ' for the question '

In [8]:
def evaluate(prompt):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response['choices'][0].text.strip()

In [9]:
def play_audio(path, player='os'):
#     tune = mp.newMedia(path)
#     mp.currentPlaylist.appendItem(tune)
#     mp.controls.play()
#     sleep(1)
#     mp.controls.playItem(tune)
    
    if player == 'vlc':
        p = vlc.MediaPlayer(path)
        p.audio_set_volume(100)
        p.play()
        sleep(1.5)
        duration = p.get_length() / 1000
        sleep(duration)
        
    else:
        os.system(path)
        sleep(3)


In [10]:
for question in questions:
    print('==========================================================')
    print('Question : ', question['q_txt'])
    print('\t Task - Converting Text to Audio using Google TTS')
    convert_text_to_audio(question)
    print('\t Playing converted audio file ')
    play_audio(question['q_audio_f_name'])
    sleep(1)
    print('Loading Answer audio file from system')
    play_audio(question['ans_audio_f_name'], player='vlc')
    print('\t Task - Converting ans audio to text using chatgpt')
    transcription = convert_audio_to_text(question)
    print('\t ChatGPT converted ans text : ', transcription)
    print('\n')
    question['ans_txt'] = transcription
    prompt = prefix +'"' +transcription +'"'+ interfix +'"'+ question['q_txt'] +'"'
    print('----------------------------------------------------------')
    print('                  ChatGPT Evaluation                      ')
    print('----------------------------------------------------------')
    print('Evaluation Scores:')
    eval_str = evaluate(prompt)
    for resp_str in eval_str.split('\n\n'):
        print(f"{resp_str}")


Question :  What are your strengths and weaknesses?
	 Task - Converting Text to Audio using Google TTS
	 Playing converted audio file 
Loading Answer audio file from system
	 Task - Converting ans audio to text using chatgpt
	 ChatGPT converted ans text :  I have several strengths namely I am patient, committed, honest and self-motivated. I am the biggest dreamer I have ever come across. I forgive easily and hate to keep grudges in my heart. My greatest weakness is that I don't like getting interrupted when I am seriously into something.


----------------------------------------------------------
                  ChatGPT Evaluation                      
----------------------------------------------------------
Evaluation Scores:
Context: The person is answering the question "What are your strengths and weaknesses?"
Answer: 8/10. The answer is comprehensive and provides multiple strengths and weaknesses. The person is honest about their strengths and weaknesses and provides a clear s